## Advise re identity architecture

Initialize

In [1]:
#r "nuget: Microsoft.SemanticKernel, 1.0.1"
#r "nuget: Microsoft.SemanticKernel, 1.0.1"
#r "nuget: Microsoft.SemanticKernel.Planners.Handlebars, 1.0.1-preview"
#r "nuget: Microsoft.SemanticKernel.Connectors.OpenAI, 1.0.1"

#r "nuget: System.Linq.Async, 6.0.1"

#!import ../config/Settings.cs

using Microsoft.SemanticKernel;

using Kernel = Microsoft.SemanticKernel.Kernel;

var (useAzureOpenAI, model, azureEndpoint, apiKey, bingApiKey, orgId) = Settings.LoadFromFile();
var kernel = Microsoft.SemanticKernel.Kernel.CreateBuilder()
            .AddAzureOpenAIChatCompletion(
                model,   // deployment name
                azureEndpoint, // Azure OpenAI Endpoint
                apiKey)      // Azure OpenAI Key
            .Build();        

Installed Packages Microsoft.SemanticKernel, 1.0.1 Microsoft.SemanticKernel.Connectors.OpenAI, 1.0.1 Microsoft.SemanticKernel.Planners.Handlebars, 1.0.1-preview System.Linq.Async, 6.0.1

Define/load plugin to facilitate use of chat history

In [3]:
using System.IO;

kernel.Plugins.Clear();

#pragma warning disable SKEXP0052
var pluginPath = Path.Combine(System.IO.Directory.GetCurrentDirectory(), "..", "plugins");
kernel.CreatePluginFromPromptDirectory(Path.Combine(pluginPath, "IdentityPlugin"));


Run

In [7]:
using Microsoft.SemanticKernel.Planning.Handlebars;

#pragma warning disable SKEXP0060
var planner = new HandlebarsPlanner(new HandlebarsPlannerOptions() { AllowLoops = true });
var goal = "Application is for my own employees.";
var plan = await planner.CreatePlanAsync(kernel, goal);
Console.WriteLine("Plan:\n");
Console.WriteLine(plan.ToString());

var resp = await plan.InvokeAsync(kernel);
Console.WriteLine("Result:\n");
Console.WriteLine(resp);

// Should be able to save and re-load a plan: https://devblogs.microsoft.com/semantic-kernel/migrating-from-the-sequential-and-stepwise-planners-to-the-new-handlebars-and-stepwise-planner/

Error: Microsoft.SemanticKernel.KernelException: [InvalidTemplate] Could not find the plan in the results
   at Microsoft.SemanticKernel.Planning.Handlebars.HandlebarsPlanner.CreatePlanCoreAsync(Kernel kernel, String goal, CancellationToken cancellationToken)
   at Microsoft.SemanticKernel.Planning.PlannerInstrumentation.CreatePlanAsync[TPlanner,TPlan](Func`5 createPlanAsync, TPlanner planner, Kernel kernel, String goal, ILogger logger, CancellationToken cancellationToken)
   at Submission#9.<<Initialize>>d__0.MoveNext()
--- End of stack trace from previous location ---
   at Microsoft.CodeAnalysis.Scripting.ScriptExecutionState.RunSubmissionsAsync[TResult](ImmutableArray`1 precedingExecutors, Func`2 currentExecutor, StrongBox`1 exceptionHolderOpt, Func`2 catchExceptionOpt, CancellationToken cancellationToken)